## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2024-03-07 23:23:33.540247: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition','Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
Xlist = ['Education','Age','DistanceFromHome','JobSatisfaction','OverTime','StockOptionLevel',
         'WorkLifeBalance','YearsAtCompany','YearsSinceLastPromotion','NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[Xlist]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
display(X_df['OverTime'].value_counts())
X_train['OverTime'] = X_train['OverTime'].str.replace('No','0')
X_train['OverTime'] = X_train['OverTime'].str.replace('Yes','1')
X_test['OverTime'] = X_test['OverTime'].str.replace('No','0')
X_test['OverTime'] = X_test['OverTime'].str.replace('Yes','1')

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.fit_transform(X_test)

In [8]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')#.set_output(transform="pandas")

# Encoded columns
train_encoded = ohe.fit_transform(y_train[['Department']])
train_encoded

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [9]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = ohe.transform(y_train[['Department']])
y_test_department_encoded = ohe.transform(y_test[['Department']])
                                       
#print(y_train_department_encoded)
print(y_test_department_encoded)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [10]:
# Create a OneHotEncoder for the Attrition column
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = ohe.transform(y_train[['Attrition']])
y_test_attrition_encoded = ohe.transform(y_test[['Attrition']])

#print(y_train_attrition_encoded)
print(y_test_attrition_encoded)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 

## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
X_train_column_length = len(X_train.columns)

# Create the input layer
input = layers.Input(shape=(X_train_column_length,))
# Create at least two shared layers
shared1 = layers.Dense(units=64, activation="sigmoid", name='shared1')(input)
shared2 = layers.Dense(units=128, activation="sigmoid", name='shared2')(shared1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=32, activation="relu", name='department_hidden')(shared2)
# Create the output layer
department_output = layers.Dense(units=3, activation="sigmoid", name='department_output')(department_hidden)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=32, activation="relu", name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(units=2, activation="sigmoid", name='attrition_output')(attrition_hidden)

In [14]:
# Create the model
nn_model = Model(inputs=input,outputs=[department_output,attrition_output])

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Summarize the model
nn_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [15]:
# Train the model
nn_model.fit(X_train_scaled, [y_train_department_encoded,y_train_attrition_encoded], epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 1ms/step - loss: 1.1412 - department_output_loss: 0.6564 - attrition_output_loss: 0.4847 - department_output_accuracy: 0.5644 - attrition_output_accuracy: 0.8285
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: 0.9677 - department_output_loss: 0.5226 - attrition_output_loss: 0.4451 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8358
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 0.9314 - department_output_loss: 0.4896 - attrition_output_loss: 0.4418 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8358
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 0.9260 - department_output_loss: 0.4878 - attrition_output_loss: 0.4382 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8358
Epoch 5/100
35/35 [==============================] - 0s 1ms/step - loss: 0.9221 - department_output_loss: 0.4874 - attrition_out

35/35 [==============================] - 0s 1ms/step - loss: 0.8469 - department_output_loss: 0.4847 - attrition_output_loss: 0.3622 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8548
Epoch 39/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8435 - department_output_loss: 0.4843 - attrition_output_loss: 0.3592 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8575
Epoch 40/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8443 - department_output_loss: 0.4840 - attrition_output_loss: 0.3603 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8539
Epoch 41/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8416 - department_output_loss: 0.4840 - attrition_output_loss: 0.3576 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8603
Epoch 42/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8423 - department_output_loss: 0.4835 - attrition_output_loss

35/35 [==============================] - 0s 1ms/step - loss: 0.8298 - department_output_loss: 0.4828 - attrition_output_loss: 0.3470 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8566
Epoch 76/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8288 - department_output_loss: 0.4824 - attrition_output_loss: 0.3464 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8521
Epoch 77/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8276 - department_output_loss: 0.4818 - attrition_output_loss: 0.3458 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8593
Epoch 78/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8259 - department_output_loss: 0.4819 - attrition_output_loss: 0.3440 - department_output_accuracy: 0.6443 - attrition_output_accuracy: 0.8630
Epoch 79/100
35/35 [==============================] - 0s 1ms/step - loss: 0.8259 - department_output_loss: 0.4823 - attrition_output_loss

In [34]:
# Evaluate the model with the testing data
loss, department_output_loss, attrition_output_loss, department_output_accuracy, attrition_output_accuracy = nn_model.evaluate(X_test_scaled,[y_test_department_encoded,y_test_attrition_encoded],verbose=2)

12/12 - 0s - loss: 0.8102 - department_output_loss: 0.4614 - attrition_output_loss: 0.3488 - department_output_accuracy: 0.6821 - attrition_output_accuracy: 0.8668 - 32ms/epoch - 3ms/step


In [41]:
# Print the accuracy for both department and attrition
display(f'Department predictions accuracy: {department_output_accuracy}')
display(f'Attrition predictions accuracy: {attrition_output_accuracy}')

'Department predictions accuracy: 0.6820651888847351'

'Attrition predictions accuracy: 0.866847813129425'

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 